In [18]:
%run Requirements.ipynb

In [19]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='MusicFusion'
)
cursor = conn.cursor()

Artist Details and Images

In [20]:
df = spark.sql("""
        SELECT DISTINCT Artist FROM 
        PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicFusion-Details` 
        ORDER BY Artist
    """)

In [21]:
auth_header = base64.b64encode((spotify_secrets.get("client_id")+':'+spotify_secrets.get("client_secret")).encode()).decode()
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_header}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(spotify_auth_url, headers=headers, data=data)
response_data = response.json()['access_token']

headers = {
    "Authorization": f"Bearer {response_data}"
}

response = requests.get(spotify_url, headers=headers)
spotify_response_data=response.json()

In [22]:
artist_names = [row['Artist'] for row in df.collect()]

artist_details = []

for artist_name in artist_names:
    spotify_api_url = f'https://api.spotify.com/v1/search?q={artist_name}&type=artist'
    headers = {
        'Authorization': f'Bearer {response_data}'
    }

    try:
        response = requests.get(spotify_api_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            artists = data.get('artists', {}).get('items', [])

            for artist_data in artists:
                if artist_data.get('name').lower() == artist_name.lower():
                    artist_details.append({
                        'name': artist_data.get('name'),
                        'id': artist_data.get('id'),
                        'image_url': artist_data.get('images')[0].get('url') if artist_data.get('images') else None,
                        'followers': artist_data.get('followers', {}).get('total'),
                        'popularity': artist_data.get('popularity')
                    })
                    break

    except requests.exceptions.RequestException as e:
        print(f"Request failed for artist '{artist_name}': {e}")

artist_df = spark.createDataFrame(artist_details)

artist_df.write.mode('overwrite').format('parquet').save('Mart/ArtistDetails')

In [23]:
df = pd.read_parquet('Mart/ArtistDetails')

columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS ArtistDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS ArtistDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO ArtistDetails VALUES {values};"
    cursor.execute(insert_query)
conn.commit()

In [24]:
df = spark.sql(
    """WITH AppleCTE AS (
        SELECT DISTINCT `Artist ID` AS AppleArtistID, `Artist Name` AS AppleArtist FROM 
        PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic`
    ),
    ArtistDetailsCTE AS (
        SELECT CAST(followers AS INT),
    id AS SpotifyArtistID,
    name AS SpotifyArtist,
    CAST(popularity AS INT) AS Popularity,
    image_url FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/ArtistDetails`
    ),
    YoutubeMusicCTE AS (
        SELECT DISTINCT `Artist Name` AS YoutubeArtist,`Artist IDs` AS YoutubeArtistID FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/YoutubeMusic`
    )
    SELECT *
    FROM ArtistDetailsCTE AS AD 
    INNER JOIN AppleCTE AS ACTE 
    ON LTRIM(RTRIM(LOWER(AD.SpotifyArtist))) = LTRIM(RTRIM(LOWER(ACTE.AppleArtist)))
    LEFT JOIN YoutubeMusicCTE AS YM 
    ON LTRIM(RTRIM(LOWER(AD.SpotifyArtist))) = LTRIM(RTRIM(LOWER(YM.YoutubeArtist)))
    """
)

In [25]:
df.write.mode('overwrite').format('parquet').save('Mart/ArtistDetailsFinal')

In [29]:
data = pd.read_parquet('Mart/ArtistDetailsFinal')

In [30]:
drop_table_query = "DROP TABLE IF EXISTS ArtistDetailsFinal;"
cursor.execute(drop_table_query)

columns = ", ".join([f"`{col}` INT" if col == 'popularity' else f"`{col}` VARCHAR(1024)" for col in data.columns])
create_table_query = f"CREATE TABLE IF NOT EXISTS ArtistDetailsFinal ({columns});"
cursor.execute(create_table_query)

insert_query = f"INSERT INTO ArtistDetailsFinal ({', '.join([f'`{col}`' for col in data.columns])}) VALUES ({', '.join(['%s' for _ in data.columns])})"

for index, row in data.iterrows():
    values = tuple(None if pd.isnull(value) else value for value in row.values)
    cursor.execute(insert_query, values)

In [31]:
conn.commit()